# Experiment with randomly selected parametered multiple runs for all strategies

In [1]:
symbols = ['BTC', 'ETH', 'BNB', 'XRP']

In [2]:
import random

import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("./data/BTC_6h.csv")
df.head()

,open_time,open,high,low,close,close_time,number_of_trades
0,2017-08-17 00:00:00,4261.48,4328.69,4261.32,4315.32,2017-08-17 05:59:59.999,273
1,2017-08-17 06:00:00,4330.29,4485.39,4287.41,4427.30,2017-08-17 11:59:59.999,309
2,2017-08-17 12:00:00,4436.06,4485.39,4235.45,4289.24,2017-08-17 17:59:59.999,1448
3,2017-08-17 18:00:00,4289.24,4369.69,4200.74,4285.08,2017-08-17 23:59:59.999,1397
4,2017-08-18 00:00:00,4285.08,4340.62,4134.61,4313.56,2017-08-18 05:59:59.999,1670


# Random all in buyer/seller

In [4]:
bank = 1000
wallet = 0

action = "buy"

for i in range(len(df)):
    row_data = df.iloc[i]
    price_to_buy_or_sell = (row_data["open"] + row_data["close"] + row_data["low"] + row_data["high"]) / 4

    if random.random() < 0.5:
        if action == "buy" and bank > 0:
            wallet = bank / price_to_buy_or_sell
            bank = 0
            action = "sell"
        elif action == "sell" and wallet > 0:
            bank = wallet * price_to_buy_or_sell
            wallet = 0
            action = "buy"

    if i%1000 == 0 and i>0:
        print("Total value:", bank + wallet * price_to_buy_or_sell)

bank = bank + wallet * price_to_buy_or_sell

print("last day:", bank)

Total value: 1452.9370128922692
Total value: 1156.5694817519504
Total value: 1353.0748075317658
Total value: 1449.401319891367
Total value: 2796.713719016595
Total value: 3354.411184613516
Total value: 2818.2293238325997
Total value: 2590.111051084498
Total value: 2860.5184035187576
Total value: 3459.4335664506502
Total value: 4508.765434779719
last day: 5105.964304246238


# Random portion buyer/seller

In [5]:
bank = 1000
wallet = 0

action = "buy"

for i in range(len(df)):
    row_data = df.iloc[i]
    price_to_buy_or_sell = (row_data["open"] + row_data["close"] + row_data["low"] + row_data["high"]) / 4

    if random.random() < 0.5:
        portion_to_use = random.random()

        if action == "buy" and bank > 0:
            wallet = wallet + (bank*portion_to_use) / price_to_buy_or_sell
            bank = bank - bank*portion_to_use
            action = "sell"
        elif action == "sell" and wallet > 0:
            bank = bank + wallet * portion_to_use * price_to_buy_or_sell
            wallet = wallet - wallet*portion_to_use
            action = "buy"

    if i%1000 == 0 and i>0:
        print("Total value:", bank + wallet * price_to_buy_or_sell)

bank = bank +  wallet * price_to_buy_or_sell

print("last day:", bank)

Total value: 2761.286596488929
Total value: 1576.4297310531965
Total value: 2831.8534874751986
Total value: 2714.1483361844294
Total value: 4277.715944727584
Total value: 3990.0381111698807
Total value: 4055.2094323115434
Total value: 3851.4093788055316
Total value: 4071.9122903807183
Total value: 6077.802287539886
Total value: 7865.749131352682
last day: 8696.80381297911


# holder buyer/seller

In [6]:
bank = 1000
wallet = 0
set_buying_period = 24
prev_price_to_buy_or_sell = np.inf
buying_period = set_buying_period
action = "buy"

for i in range(len(df)):
    row_data = df.iloc[i]
    price_to_buy_or_sell = (row_data["open"] + row_data["close"] + row_data["low"] + row_data["high"]) / 4

    # if it's time to buy, and we have money
    if action == "buy" and bank > 0:
        wallet = bank / row_data["low"]
        bank = 0
        action = "sell"
        prev_price_to_buy_or_sell = price_to_buy_or_sell
        buying_period = set_buying_period

    # either we bought and now waiting for hold times to pass
    # or we are waiting for
    if buying_period != 0:
        buying_period -= 1
        continue

    # if it's time to sell, and we have coin, and we can profit from selling
    if action == "sell" and wallet > 0 and price_to_buy_or_sell > prev_price_to_buy_or_sell:
        bank = wallet * row_data["high"]
        wallet = 0
        action = "buy"
        buying_period = set_buying_period
    
    if i%1000 == 0 and i>0:
        print("Total value:", bank + wallet * price_to_buy_or_sell)

bank = bank +  wallet * price_to_buy_or_sell

print("last day:", bank)

Total value: 3529.7274676095
Total value: 1394.4998528342946
Total value: 3898.052319860154
Total value: 3601.8682477756206
Total value: 21035.496710833668
Total value: 32695.921348260676
Total value: 24264.848361384462
Total value: 18854.255094670316
Total value: 23292.870789883367
Total value: 64460.723081294476
Total value: 107454.65384366391
last day: 119775.25374975525


# Martingale betting

In [7]:
import numpy as np

bank = 1000  # Starting money
wallet = 0   # Amount of coin
set_holding_period = 10
base_bet = 10  # Initial bet
current_bet = base_bet
prev_price_to_buy = np.inf
holding_period = 0

action = "buy"

for i in range(len(df)):
    row_data = df.iloc[i]
    price_to_buy_or_sell = (row_data["open"] + row_data["close"] + row_data["low"] + row_data["high"]) / 4

    # Handle buy logic
    if action == "buy" and bank >= current_bet:
        coins_bought = current_bet / price_to_buy_or_sell
        wallet += coins_bought
        bank -= current_bet
        prev_price_to_buy = price_to_buy_or_sell
        holding_period = set_holding_period
        action = "sell"
    
    # Wait for the holding period to pass
    elif action == "sell":
        holding_period -= 1
        if holding_period > 0:
            continue
        else:
            # Holding period is over; decide whether to sell
            coin_value_now = wallet * price_to_buy_or_sell
            cost_basis = coins_bought * prev_price_to_buy

            profit = coin_value_now - cost_basis

            if profit >= 0:
                pass  # Reset to base bet
            else:
                # Double the previous bet or bet enough to cover the loss
                current_bet = current_bet * 2
                if current_bet > bank:
                    bank = bank + wallet * price_to_buy_or_sell
                    wallet = 0
                    if current_bet > bank:
                        current_bet = base_bet
            action = "buy"

    if i%1000 == 0 and i>0:
        print("Total value:", bank + wallet * price_to_buy_or_sell)

bank = bank +  wallet * price_to_buy_or_sell

print("last day:", bank)

Total value: 1590.930290086436
Total value: 634.5827977081411
Total value: 1755.8993764700456
Total value: 1623.2414374941782
Total value: 6227.278863469632
Total value: 7226.562565738126
Total value: 4955.803309391671
Total value: 3852.98454515521
Total value: 4757.689156020951
Total value: 10748.895636937128
Total value: 14310.093870465354
last day: 15949.722577974719


# Dollar cost averaging

In [8]:
bank = 1000
wallet = 0
period = 1000
periodic_investment_budget = 1000 / (len(df) / period)

for i in range(len(df)):
    row_data = df.iloc[i]
    price_to_buy_or_sell = (row_data["open"] + row_data["close"] + row_data["low"] + row_data["high"]) / 4

    if i % period == 0 and bank >= periodic_investment_budget:
        # Invest the fixed amount
        quantity_bought = periodic_investment_budget / price_to_buy_or_sell
        wallet += quantity_bought
        bank -= periodic_investment_budget
    
    if i%1000 == 0 and i>0:
        print("Total value:", bank + wallet * price_to_buy_or_sell)

bank = bank +  wallet * price_to_buy_or_sell

print("last day:", bank)

Total value: 1106.989828835646
Total value: 934.8789336353772
Total value: 1296.0348253712634
Total value: 1246.563675378751
Total value: 3216.833653668622
Total value: 3658.7376136237663
Total value: 2626.6308146874712
Total value: 2105.5846387856645
Total value: 2553.9245417821608
Total value: 5634.967943122918
Total value: 7495.788235219159
last day: 8352.537612917842


## SMA crossover

In [9]:
bank = 1000
wallet = 0
action = "buy"  # Initial action can be to buy if there's bank

short_window = 10
long_window = 30

# Calculate Simple Moving Averages
df['SMA_short'] =((df["open"] + df["close"] + df["low"] + df["high"]) / 4).rolling(window=short_window, min_periods=1).mean()
df['SMA_long'] = ((df["open"] + df["close"] + df["low"] + df["high"]) / 4).rolling(window=long_window, min_periods=1).mean()

for i in range(len(df)):
    row_data = df.iloc[i]
    price_to_buy_or_sell = (row_data["open"] + row_data["close"] + row_data["low"] + row_data["high"]) / 4

    # Check for crossover signals (avoiding the first few periods where SMAs are still calculating)
    if i > long_window:
        previous_row = df.iloc[i-1]
        short_above_long = previous_row['SMA_short'] < previous_row['SMA_long'] and row_data['SMA_short'] > row_data['SMA_long']
        short_below_long = previous_row['SMA_short'] > previous_row['SMA_long'] and row_data['SMA_short'] < row_data['SMA_long']

        if short_above_long and action == "buy" and bank > 0:
            # Buy with all available bank
            wallet = bank / price_to_buy_or_sell
            bank = 0
            action = "sell"

        elif short_below_long and action == "sell" and wallet > 0:
            # Sell all available wallet
            bank = wallet * price_to_buy_or_sell
            wallet = 0
            action = "buy"

    if i%1000 == 0 and i>0:
        print("Total value:", bank + wallet * price_to_buy_or_sell)

bank = bank +  wallet * price_to_buy_or_sell

print("last day:", bank)

Total value: 2177.2180878903446
Total value: 1563.444564669182
Total value: 3615.4011714832723
Total value: 3558.1268154996355
Total value: 8728.868800347946
Total value: 8533.817381564153
Total value: 8276.775823994705
Total value: 7809.630308891603
Total value: 8128.153728418645
Total value: 18110.860613154673
Total value: 19299.968842766597
last day: 20200.549178107693


## Volatility

In [10]:
bank = 1000
wallet = 0
action = "buy"  # Initial action

# Calculate ATR
high_low = df['high'] - df['low']
high_close_prev = abs(df['high'] - df['close'].shift(1))
low_close_prev = abs(df['low'] - df['close'].shift(1))
tr = pd.concat([high_low, high_close_prev, low_close_prev], axis=1).max(axis=1)
df['ATR'] = tr.rolling(window=16, min_periods=1).mean()

for i in range(len(df)):
    row_data = df.iloc[i]
    price_to_buy_or_sell = (row_data["open"] + row_data["close"] + row_data["low"] + row_data["high"]) / 4
    atr = row_data["ATR"]

    # Define dynamic volatility thresholds based on the mean ATR
    mean_atr = df['ATR'].iloc[:i].mean() if i > 0 else atr  # Use historical mean

    if mean_atr > 0:
        low_volatility_threshold = mean_atr * 1.25
        high_volatility_threshold = mean_atr * 2.5
    else:
        low_volatility_threshold = 0
        high_volatility_threshold = float('inf')

    # Buy logic: After a period of low volatility, expecting a breakout
    if atr < low_volatility_threshold and action == "buy" and bank > 0:
        # Buy with all available bank
        wallet = bank / price_to_buy_or_sell
        bank = 0
        action = "sell"

    # Sell logic: After a period of high volatility, expecting consolidation
    elif atr > high_volatility_threshold and action == "sell" and wallet > 0:
        # Sell all available wallet
        bank = wallet * price_to_buy_or_sell
        wallet = 0
        action = "buy"

    if i%1000 == 0 and i>0:
        print("Total value:", bank + wallet * price_to_buy_or_sell)

bank = bank +  wallet * price_to_buy_or_sell

print("last day:", bank)

Total value: 1931.3427474183786
Total value: 763.0212818871009
Total value: 2581.035502603903
Total value: 1917.2368346955238
Total value: 4275.9792574807
Total value: 4275.9792574807
Total value: 2699.329183579358
Total value: 2162.4229310809606
Total value: 2671.494454373215
Total value: 5857.458682635003
Total value: 5856.942973097878
last day: 6339.397128457123


# Trend following

In [11]:
bank = 1000
wallet = 0
action = "buy"  # Initial action

for i in range(len(df)):
    price_to_buy_or_sell = df['close'].iloc[i]

    # Check for buy signal
    if action == "buy" and bank > 0 and i >= 16:
        previous_price = df['close'].iloc[i - 16]
        percentage_change = (price_to_buy_or_sell - previous_price) / previous_price
        if percentage_change >= 0.05:
            # Buy with all available bank
            wallet = bank / price_to_buy_or_sell
            bank = 0
            action = "sell"

    # Check for sell signal
    elif action == "sell" and wallet > 0 and i >= 5:
        previous_price = df['close'].iloc[i - 5]
        percentage_change = (price_to_buy_or_sell - previous_price) / previous_price
        if percentage_change <= -0.05:
            # Sell all available wallet
            bank = wallet * price_to_buy_or_sell
            wallet = 0
            action = "buy"

    if i%1000 == 0 and i>0:
        print("Total value:", bank + wallet * price_to_buy_or_sell)

bank = bank +  wallet * price_to_buy_or_sell

print("last day:", bank)

Total value: 2477.047358750303
Total value: 1859.4048751134128
Total value: 3720.06788984578
Total value: 3207.0488205919555
Total value: 8511.986250730655
Total value: 7125.101917530751
Total value: 6283.895963023996
Total value: 6237.217517698899
Total value: 5657.442783599988
Total value: 10579.746261603814
Total value: 14144.277561280182
last day: 12740.26832677218


# Toast Bread

In [ ]:
"""

Toast Bread strategy is a strategy that I came up with, which utilizes the limited future insight that keeps flowing when time progresses. It consists of 2 stages: stage A where the agent creates a plan and stage B/C where the agent acts upon the newly came future insight. Before explaining the underlying logic, first lets talk about a scenario where our trained models make predictions for the 8 upcoming time-series data with 6 hours intervals. It predicts 4 features for each interval: open, close, low and high. So our future-insight is in this format: [d1.0-6, d1.6-12, d1.12-18, d1.18-24, d2.0-6, d2.6-12, d2.12-18, d2.18-24] where di.s-e means ith day's [open, close, low, end] information for the interval between start hour s and end hour h.
And these are the terms for having common ground on the explanations.

'slice' is a 6 hour interval, it contains an open and a close price that is at the start and end of the 6 hour interval. And it contains low and high prices which are the lowest and highest prices the coin has achieved in this interval. We don't have any time information about these two. 'di.s-e' is a slice.

'sandwich' is consecutive slices back to back. A single slice is not a sandwich.

And we set all of our buy and sell orders with an error margin. Lets say the coin comes 100$ close to our prediction price, then we take the action, and if it doesn't we don't do anything. Below is how the strategy works step by step:


A. 1- find the best profiting sandwich, that has the profit margin of (highest-lowest), where 'lowest' is the low price of the slice that is at the beginning of the sandwich; and 'highest' is the high price of the slice that is at the end of the sandwich.

A. 2- create a buy-sell order to buy the coin at sandwich's lowest and sell at sandwich's highest

A. 3- remove this sandwich from the data

A. 4- continue doing A.1-2-3 steps at remaining data (but we approach them as seperate time-series data since removed sandwich split them into two unrelated parts) till there is no profitable sandwich left.

A. 5- for all remaining slices in our data, do one of these:

A. 5.1- create a buy order at open price if we can sell at close price or high price with profit

A. 5.2- create a buy order at low price if we can sell at close price with profit


And now we have a plan with a list of set buy-sell orders for the sandwichs (also could be empty), and also buy-sell orders for the slices (also could be empty). If we have some orders for the current interval that we are in right now, we do them. Now we go to next time interval. Since we enter the new interval, we now got next predictions from the models. It covers again upcoming 8 intervals so there is a 7 interval overlap but we select to trust newly came predictions rather than old ones. After entering the next interval, now we can be in two stages, first easier stage B:

B. 1- we dont have any ongoing plan (meaning we are not inbetween some buy-sell order of a sandwich from stage A. We either did a slice buy-sell order, or didn't take any actions at all)

B. 2- if so, then we go back to stage A where we had no plan, and do A.1-2-3-4-5 buy-sell order setting steps again with the newly came predictions.


or harder stage C:


C. 1- we have some ongoing plan (meaning we had a set buy-sell order which we completed the buy part and now waiting for the sell part)

C. 2- we trust the newly came slices more than our past data so we discard all plans other than the ongoing one because.

C. 3- and we check the state of our ongoing plan, which can be updated in three different way as below:

C. 4.1- if we have a better selling point at a newly predicted slice's high price on the timeline, we change our current ongoing plan's selling point to that point, and discard any prediction that comes before that. do the steps A-1-2-3-4-5 for the remaining data

C. 4.2- if we don't have a better selling point at a newly came slice further on the timeline and our previous selling slice's high price is still valid, continue with our plan and do the A-1-2-3-4-5 with remaining data

C. 4.3, if we don't have a better selling point at further on the timeline and our previous selling slice's high price is now invalid (which means that the newly came predictions contradict with the past ones), find the highest selling point that we can find, set our ongoing plan's selling point to that slice's high price to get away with lowest money loss. and do the A-1-2-3-4-5 with remaining ones


This method is named after a toast bread since it's easier to visualize this 6 hour intervals with an image that has a width rather than a timeline of the data (it's confusing to think about our open, close, low, high features when first 2 has a timestamp and second two's occurance time is not knoww which means we can't set a buy-sell order like buying at di.12-18.low and selling at di.12-18.high since high might have occured before the low). Also it helps with the easy-to-remember non-terminologic words by naming the intervals as slices and back to back intervals as sandwiches.

This way we can show the effects of 'knowing predicted future' on the agent profits compared to 'unknown future' and also 'ground-truth future'.

"""

"\nor harder stage C:\nC. 1- you have some ongoing plan (meaning you had set a buy-sell order which you completed the buy part and now waiting for the sell part [end of the sandwich was in the last 4 slices while doing the planning])\nC. 2- now you discard all plans other than the ongoing one because we trust the newly came slices more than our past data.\nC. 3- and you check the state of your ongoing plan, which can be updated in three different way as below:\nC. 4.1- if you have a better selling point at a newly predicted slice's highest further on the timeline, change your current ongoing plan's selling point to that point, and discard any slice that comes before that. do the steps A-1-2-3-4-5 for the remaining slices\nC. 4.2- if you don't have a better selling point at a newly came slice further on the timeline and your previous selling slice's highest is still valid, continue with your plan and do the A-1-2-3-4-5 with remaining slices\nC. 4.3, if you don't have a better selling po

In [4]:
def get_orders(oclh):
    # calculate each (buy at low, sell at an upcoming high) profit
    profits = []
    for l in range(oclh.shape[1]):
        for h in range(l + 1, oclh.shape[1]):
            profit = oclh[3][h] - oclh[2][l]
            if profit > 0:
                profits.append(((l, h), (oclh[2][l], oclh[3][h]), profit))

    # calculate sandwiches
    sandwiches = []
    while len(profits) > 0:
        max_b_s_p_l_h = max(profits, key=lambda item: item[2])
        if max_b_s_p_l_h[2] > 0:
            sandwiches.append(max_b_s_p_l_h)
            profits = [x for x in profits if x[0][1] < max_b_s_p_l_h[0][0] or x[0][0] > max_b_s_p_l_h[0][1]]

    # collect remaining slices
    slices = [x for x in range(oclh.shape[1]) if x not in sum(([list((range(x[0][0], x[0][1]+1))) for x in sandwiches]), [])]

    orders = sandwiches

    # create buy sell orders from slices and collect all orders in a list
    for s in slices:
        slice_ = oclh[:, s]
        if slice_[3] - slice_[0] > 0 or slice_[1] - slice_[2] > 0:
            # if high-open is bigger than close-low
            if slice_[3] - slice_[0] > slice_[1] - slice_[2]:
                orders.append(((s, s), (slice_[0], slice_[3]), slice_[3]-slice_[0])) # buy at open, sell at high
            else:
                orders.append(((s, s), (slice_[2], slice_[1]), slice_[1]-slice_[2])) # buy at low, sell at close

    # sorted orders in the format of [(buy_day_index, sell_day_index), (buy_price, sell_price), profit]
    orders = sorted(orders, key=lambda x:x[0][0])
    orders = [{"buy_day":x[0][0], "sell_day":x[0][1], "buy_price":x[1][0], "sell_price":x[1][1], "profit":x[2]} for x in orders]
    return orders

In [ ]:
bank = 1
wallet = 0
output_window = 8
ongoing_order = None
sold_at_first_order_open_to_buy_later_for_first_order = False
buys, sells = [], []
prediction_margin = 50 # use this like "I am expecting the price to be x, so I'll buy at x+50 and sell at x-50"

for i in range(len(df)-output_window-1):
    row_data = df.iloc[i:i+output_window]
    open_, close, low, high = row_data["open"], row_data["close"], row_data["low"], row_data["high"]

    # create timeline matrix
    oclh = np.vstack((np.array(open_), np.array(close), np.array(low), np.array(high)))
    price_to_buy_or_sell = np.mean(oclh[:, 0])

    if ongoing_order is None:
        orders = get_orders(oclh)
        #print("No ongoing order, here are newly calculated orders:", orders)

        if len(orders) > 0 and orders[0]["buy_day"] == 0:
            ongoing_order = orders[0]
            if bank > 0:
                # print(f'BANK:{bank}, WALLET:{wallet} buying at {ongoing_order["buy_price"]}')
                wallet += bank / ongoing_order["buy_price"]
                bank = 0
                # print(f'BANK:{bank}, WALLET:{wallet}')
                buys.append((i, ongoing_order["buy_price"]))
        else:
            continue
        
        if ongoing_order["sell_day"] == 0:
            if wallet > 0:
                # print(f'BANK:{bank}, WALLET:{wallet} selling at {ongoing_order["sell_price"]}')
                bank += wallet * ongoing_order["sell_price"]
                wallet = 0
                # print(f'BANK:{bank}, WALLET:{wallet}')
                sells.append((i, ongoing_order["sell_price"]))
            ongoing_order = None
        
        if ongoing_order:
            ongoing_order["buy_day"] -= 1
            ongoing_order["sell_day"] -= 1

    else:
        #print("Ongoing order:", ongoing_order)
        first_order = get_orders(oclh)[0]
        #print("First order:", first_order)

        if first_order is not None:
            # ongoing order eats the first order if it's profit will get bigger
            if first_order["buy_price"] > ongoing_order["sell_price"]:
                ongoing_order = {"buy_day": ongoing_order["buy_day"], "sell_day": first_order["sell_day"],
                                  "buy_price": ongoing_order["buy_price"],"sell_price": first_order["sell_price"],
                                  "profit": first_order["sell_price"] - ongoing_order["buy_price"]}
                #print("Ongoing order ate the first order:", ongoing_order)
            else: # there is more profit chance at selling ongoing and then doing the first order do that
                if first_order["buy_day"] == 0 and ongoing_order["sell_day"] != 0:
                    # set the ongoing_order sell date as the open price of the first_order's buy day
                    ongoing_order = {"buy_day": ongoing_order["buy_day"], "sell_day": 0,
                                  "buy_price": ongoing_order["buy_price"],"sell_price": oclh[0, 0],
                                  "profit": oclh[0, 0] - ongoing_order["buy_price"]}
                    
                    #print("Ongoing order sell date changed because first order immediate:", ongoing_order)
                    sold_at_first_order_open_to_buy_later_for_first_order = True
                elif first_order["buy_day"] > 0:
                    # set the ongoing_order sell date as the highest that comes before first order
                    highest_day_before_first_order, highest_price_before_first_order = np.argmax(oclh[3, :first_order["buy_day"]]), max(oclh[3, :first_order["buy_day"]])
                    ongoing_order = {"buy_day": ongoing_order["buy_day"], "sell_day": highest_day_before_first_order,
                                  "buy_price": ongoing_order["buy_price"],"sell_price": highest_price_before_first_order,
                                  "profit": highest_price_before_first_order - ongoing_order["buy_price"]}
                    #print("Ongoing order sell date changed to highest before first order:", ongoing_order)

        # if we set the ongoing_order sell date as today, sell it
        if ongoing_order["sell_day"] == 0:
            if wallet > 0:
                # print(f'BANK:{bank}, WALLET:{wallet} selling at {ongoing_order["sell_price"]}')
                bank += wallet * ongoing_order["sell_price"]
                wallet = 0
                # print(f'BANK:{bank}, WALLET:{wallet}')
                sells.append((i, ongoing_order["sell_price"]))
            ongoing_order = None

        if sold_at_first_order_open_to_buy_later_for_first_order:
            sold_at_first_order_open_to_buy_later_for_first_order = False
            ongoing_order = first_order
            if bank > 0:
                # print(f'BANK:{bank}, WALLET:{wallet} buying at {ongoing_order["buy_price"]}')
                wallet += bank / ongoing_order["buy_price"]
                bank = 0
                # print(f'BANK:{bank}, WALLET:{wallet}')
                buys.append((i, ongoing_order["buy_price"]))
            ongoing_order["buy_day"] -= 1
            ongoing_order["sell_day"] -= 1

    # print("")
    # print(oclh)
    # print("")

    if i%1000 == 0 and i>0:
        print("Total value:", bank + wallet * price_to_buy_or_sell)

bank += wallet * price_to_buy_or_sell
print("last day:", bank)

Total value: 676671851393.8557
Total value: 2.431511834442308e+17
Total value: 2.7475109099696735e+23
Total value: 5.2393299123467266e+29
Total value: 4.3183656300700035e+35
Total value: 1.5390800979380545e+44
Total value: 1.4236628447986207e+50
Total value: 1.1474788786691262e+55
Total value: 5.354053150629924e+58
Total value: 2.7564156497307087e+63
Total value: 1.3743479854558817e+68
last day: 2.489229946411521e+69


In [23]:
import pandas as pd
import plotly.graph_objects as go
import matplotlib.pyplot as plt

In [ ]:
plot_len = 100

df2 = pd.read_csv("./data/BTC_6h.csv")

# Plotting
plt.figure(figsize=(12, 6))
plt.plot(range(len(df2['open_time'][:plot_len])), df2['open'][:plot_len], color='orange')
plt.plot(range(len(df2['open_time'][:plot_len])), df2['close'][:plot_len], color='blue')
plt.plot(range(len(df2['open_time'][:plot_len])), df2['low'][:plot_len], color='red')
plt.plot(range(len(df2['open_time'][:plot_len])), df2['high'][:plot_len], color='green')

# Draw buy-sell lines
for (buy_idx, buy_price), (sell_idx, sell_price) in zip(buys[:plot_len], sells[:plot_len]):
    if buy_idx > plot_len:
        break
    plt.plot([buy_idx, sell_idx], [buy_price, sell_price],
             color='green' if sell_price > buy_price else 'red', linewidth=2,marker='o')

plt.tight_layout()
plt.show()

In [ ]:
# Load the data
df2 = pd.read_csv("./data/BTC_6h.csv")
plot_len = len(df2)  # or however many you want

# Compute x-axis (time) — convert to datetime for better interactivity
df2['open_time'] = pd.to_datetime(df2['open_time'])
time = df2['open_time'][:plot_len]

# Create the figure
fig = go.Figure()

# Add price lines
fig.add_trace(go.Scatter(x=time, y=df2['open'][:plot_len], mode='lines', name='Open', line=dict(color='orange')))
fig.add_trace(go.Scatter(x=time, y=df2['close'][:plot_len], mode='lines', name='Close', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=time, y=df2['low'][:plot_len], mode='lines', name='Low', line=dict(color='red')))
fig.add_trace(go.Scatter(x=time, y=df2['high'][:plot_len], mode='lines', name='High', line=dict(color='green')))

# Add buy-sell arrows/lines
for (buy_idx, buy_price), (sell_idx, sell_price) in zip(buys, sells):
    if buy_idx >= plot_len or sell_idx >= plot_len:
        continue
    fig.add_trace(go.Scatter(
        x=[time.iloc[buy_idx], time.iloc[sell_idx]],
        y=[buy_price, sell_price],
        mode='lines+markers',
        line=dict(color='green' if sell_price > buy_price else 'red', width=2),
        marker=dict(size=6),
    ))

# Update layout for interactivity
fig.update_layout(height=600)

# Show plot
fig.show()